In [ ]:
import requests
import json
import pandas as pd

In [ ]:
api = 'your_key'
url = 'https://api.lens.org/patent/search'

In [27]:
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "query_string": {
                        "query": "\"ASO\" OR \"antisense oligonucleotides\"",
                        "fields": ["title", "abstract", "claims"],
                        "default_operator": "OR"
                    }
                }
            ]
        }
    },
    "size": 13
}

headers = {
    'Authorization': f'Bearer {api}',
    'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, data=json.dumps(query))

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code} - {response.text}")

In [42]:
# the outcome and the structure of the retrieved data
data

{'total': 10679,
 'max_score': 45.98564,
 'data': [{'lens_id': '006-252-632-930-02X',
   'jurisdiction': 'CN',
   'doc_number': '104887694',
   'kind': 'A',
   'date_published': '2015-09-09',
   'doc_key': 'CN_104887694_A_20150909',
   'docdb_id': 444324796,
   'lang': 'en',
   'biblio': {'publication_reference': {'jurisdiction': 'CN',
     'doc_number': '104887694',
     'kind': 'A',
     'date': '2015-09-09'},
    'application_reference': {'jurisdiction': 'CN',
     'doc_number': '201510280396',
     'kind': 'A',
     'date': '2015-05-27'},
    'priority_claims': {'claims': [{'jurisdiction': 'CN',
       'doc_number': '201510280396',
       'kind': 'A',
       'date': '2015-05-27',
       'sequence': 1}],
     'earliest_claim': {'date': '2015-05-27'}},
    'invention_title': [{'text': 'Antisense oligonucleotides targeting non-coding RNAs and application thereof in preparation of anti-influenza virus medicament',
      'lang': 'en'}],
    'parties': {'applicants': [{'extracted_name': 

In [21]:
# the json structure of the outcome
def print_json_structure(data, indent=0):
    if isinstance(data, dict):
        for key in data:
            print('  ' * indent + str(key))
            print_json_structure(data[key], indent + 1)
    elif isinstance(data, list) and len(data) > 0:
        print('  ' * indent + '[Array of {}]'.format(type(data[0]).__name__))
        print_json_structure(data[0], indent + 1)
    else:
        print('  ' * indent + str(type(data).__name__))

if response.status_code == 200:
    data = response.json()
    print_json_structure(data)
else:
    print(f"Error: {response.status_code} - {response.text}")

total
  int
max_score
  float
data
  [Array of dict]
    lens_id
      str
    jurisdiction
      str
    doc_number
      str
    kind
      str
    date_published
      str
    doc_key
      str
    docdb_id
      int
    lang
      str
    biblio
      publication_reference
        jurisdiction
          str
        doc_number
          str
        kind
          str
        date
          str
      application_reference
        jurisdiction
          str
        doc_number
          str
        kind
          str
        date
          str
      priority_claims
        claims
          [Array of dict]
            jurisdiction
              str
            doc_number
              str
            kind
              str
            date
              str
            sequence
              int
        earliest_claim
          date
            str
      invention_title
        [Array of dict]
          text
            str
          lang
            str
      parties
        applicants

In [40]:
data_list = []

for patent in data.get('data', []):
    lens_id = patent.get('lens_id', 'N/A')

    title = patent.get('biblio', {}).get('invention_title', [{}])[0].get('text', 'N/A')

    abstract = patent.get('abstract', [{}])[0].get('text', 'N/A')

    claims = " ".join(patent.get('claims', [{}])[0].get('claims', [{}])[0].get('claim_text', ["N/A"]))

    description = patent.get('description', {}).get('text', 'N/A')

    data_list.append({
        'Lens ID': lens_id,
        'Title': title,
        'Abstract': abstract,
        'Claims': claims,
        'Description': description
    })

df = pd.DataFrame(data_list)

In [41]:
# example of the created df
df

,Lens ID,Title,Abstract,Claims,Description
0,006-252-632-930-02X,Antisense oligonucleotides targeting non-codin...,The invention discloses antisense oligonucleot...,N/A,N/A
1,020-838-286-704-585,ANTISENSE OLIGONUCLEOTIDE COMPOSITIONS,The present invention relates to antisense oli...,1. An antisense oligonucleotide composition co...,Antisense Oligonucleotide Compositions The pre...
2,035-902-563-142-209,Antisense oligonucleotide (ASO) gene inhibitio...,Embodiments of the present invention relate ge...,N/A,N/A
3,073-575-549-953-147,ANTISENSE OLIGONUCLEOTIDE (ASO) GENE INHIBITIO...,Embodiments of the invention related generally...,N/A,N/A
4,093-656-511-507-299,ANTISENSE OLIGONUCLEOTIDES,Provided herein include conditionally activata...,"WHAT IS CLAIMED IS: 1. An oligonucleotide, com...",ANTISENSE OLIGONUCLEOTIDES CROSS-REFERENCE TO ...
5,103-752-500-267-462,Antisense oligonucleotides and uses thereof,Disclosed herein are novel single strand antis...,N/A,N/A
6,114-832-383-274-557,ANTISENSE OLIGONUCLEOTIDE (ASO) GENE INHIBITIO...,Embodiments of the invention related generally...,N/A,N/A
7,117-058-033-153-332,ANTISENSE OLIGONUCLEOTIDE (ASO) GENE INHIBITIO...,Embodiments of the invention related generally...,N/A,N/A
8,156-638-061-645-268,INHIBICION GENICA Y TRATAMIENTO MEDIANTE SECUE...,Embodiments of the invention related generally...,N/A,N/A
9,167-754-903-057-153,ANTISENSE OLIGONUCLEOTIDE (ASO) GENE INHIBITIO...,Embodiments of the invention related generally...,CLAIMS 1. A method of treating a patient diagn...,Antisense Oligonucleotide (ASO) Gene Inhibitio...
